In [1]:
# Install dependencies
!pip install -U ultralytics
!pip install opencv-python-headless
!pip install albumentations
!pip install labelImg


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.2 MB/s  0:00:00
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.3.240
    Uninstalling ultralytics-8.3.240:
      Successfully uninstalled ultralytics-8.3.240


In [2]:
import torch
from ultralytics import YOLO
import shutil
import pandas as pd


In [3]:
print("CUDA:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))
torch.cuda.empty_cache()

CUDA: True
GPU: NVIDIA GeForce RTX 3050 6GB Laptop GPU


In [9]:
DATASET_PATH = "/home/muliaandiki/project/NutriPlate/preprocessing/data/Dietary App Segment.v1i.yolov8"
DATA_YAML = f"{DATASET_PATH}/data.yaml"

# ini di ganti ganti
BASE_MODEL = "/home/muliaandiki/project/NutriPlate/preprocessing/runs/train/dietary_yolov8s/weights" 

In [11]:
model = YOLO(BASE_MODEL)

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.


In [ ]:
model.train(
    data=DATA_YAML,
    epochs=100,
    imgsz=640,
    batch=4,              
    optimizer="AdamW",
    patience=20,          
    device=0,
    project="runs/train",
    name="dietary_yolov8s",
    exist_ok=True,
    workers=4,
    verbose=True
)

In [ ]:
best_model = YOLO("runs/train/dietary_yolov8s/weights/best.pt")

metrics = best_model.val(
    data=DATA_YAML,
    split="val",
    imgsz=640,
    device=0
)

print(metrics)

In [ ]:

df = pd.DataFrame({
    "Class": metrics.names.values(),
    "Precision": metrics.box.p,
    "Recall": metrics.box.r,
    "mAP50": metrics.box.map50,
    "mAP50-95": metrics.box.map
})

df.sort_values("mAP50", ascending=False)


In [ ]:
TEST_IMAGES = f"{DATASET_PATH}/test/images"

results = best_model.predict(
    source=TEST_IMAGES,
    imgsz=640,
    conf=0.25,
    iou=0.5,
    save=True,
    device=0
)


In [ ]:
best_model.val(
    data=DATA_YAML,
    plots=True


In [ ]:
shutil.copy(
    "runs/train/dietary_yolov8s/weights/best.pt",
    "dietary_food_detector.pt"
)

In [ ]:
best_model.export(format="torchscript")
best_model.export(format="onnx")

In [ ]:
# Rebuild

In [12]:
model.train(
    data="preprocessing/data_merged/data.yaml",
    epochs=40,                 
    imgsz=640,
    batch=4,
    optimizer="AdamW",
    patience=15,
    device=0,
    project="runs/train",
    name="dietary_yolov8s_v2",
    exist_ok=False,
    workers=4
)

TypeError: model='/home/muliaandiki/project/NutriPlate/preprocessing/runs/train/dietary_yolov8s/weights' should be a *.pt PyTorch model to run this method, but is a different format. PyTorch models can train, val, predict and export, i.e. 'model.train(data=...)', but exported formats like ONNX, TensorRT etc. only support 'predict' and 'val' modes, i.e. 'yolo predict model=yolo11n.onnx'.
To run CUDA or MPS inference please pass the device argument directly in your inference command, i.e. 'model.predict(source=..., device=0)'